In [17]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import k_means

def loadData(dataDir, time = 500):
    dataset = np.load(dataDir)
    labels = dataset['arr_0'][:, :, 1:-1] # ignore the first row of labels (all 0)
    properties = dataset['arr_1']
    if time >= 0:
        x = properties[:, :, time]
        y = labels[:, :, time]
    else:
        x = np.reshape(properties, [properties.shape[0], properties.shape[1] * properties.shape[2]])
        y = np.reshape(labels, [labels.shape[0], labels.shape[1] * labels.shape[2]])
    return x, y



In [20]:
dataDir = '../data/dataset.npz'
xtrain, ytrain = loadData(dataDir, time = 200)

In [12]:
a = np.random.randint(0, 10, size = [3, 3, 3])
aa = np.reshape(a, (3, 9), order = 'F')